In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

%matplotlib inline

In [2]:
df = pd.read_json(os.path.join('input', 'train.json'))

In [3]:
def process_bands(df):
    # reclassify as numpy array
    for band in ['band_1', 'band_2']:
        df[band] = df[band].apply(np.asarray)
    # set band_3 as mean of first two bands
    df['band_3'] = (df.band_1 + df.band_2) / 2


# list of convenience
bands = ['band_1', 'band_2', 'band_3']

process_bands(df)

In [4]:
from IPython.display import display

display(df.iloc[0])

band_1        [-27.878360999999998, -27.15416, -28.668615, -...
band_2        [-27.154118, -29.537888, -31.0306, -32.190483,...
id                                                     dfd5f913
inc_angle                                               43.9239
is_iceberg                                                    0
band_3        [-27.516239499999998, -28.346024, -29.84960749...
Name: 0, dtype: object

In [ ]:
no_angle = df.inc_angle == 'na'
angles = df[~no_angle].inc_angle
angles.describe()

count     1471.0000
unique     878.0000
top         34.4721
freq        23.0000
Name: inc_angle, dtype: float64

In [31]:
for name, f in zip(['angle mean', 'angle median'], [np.mean, np.median]):
    print('{:15} = {:.5f}'.format(name, f(angles)))

angle mean      = 39.26871
angle median    = 39.50150


Especially since we are going to use normalization techniques, we choose to fill the incidence angle with the mean value in the training data. 

In [34]:
df.loc[no_angle, 'inc_angle'] = np.mean(angles)

In [35]:
train_df, valid_df = train_test_split(df, random_state=42)

In [36]:
def make_tensors(df):
    gen = (np.stack(getattr(df, band)) for band in bands)
    flat_tensors = np.stack(gen, axis=-1)
    return flat_tensors.reshape(flat_tensors.shape[0], 75, 75, 3)


train_tensors = make_tensors(train_df)
valid_tensors = make_tensors(valid_df)
print(train_tensors.shape)
print(valid_tensors.shape)

(1203, 75, 75, 3)
(401, 75, 75, 3)


In [38]:
train_inc_angle = train_df.inc_angle.values
valid_inc_angle = valid_df.inc_angle.values
print(train_inc_angle.shape)
print(valid_inc_angle.shape)

(1203,)
(401,)


In [40]:
train_targets = train_df.is_iceberg.values
valid_targets = valid_df.is_iceberg.values
print(train_targets.shape)
print(valid_targets.shape)

(1203,)
(401,)


In [41]:
from keras.preprocessing.image import ImageDataGenerator

/Users/charlie/miniconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=.05, 
    height_shift_range=.05, 
    rotation_range=30
)

datagen.fit(np.concatenate((train_tensors, valid_tensors)), augment=True, rounds=2)

In [61]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, concatenate
from keras.layers import Dropout
from keras.models import Model
from keras.utils import plot_model


def create_cnn(iceberg_predict=True):
    # inputs
    radar_img_shape = train_tensors.shape[1:]
    radar_input = Input(shape=radar_img_shape, name='radar_image')
    aux_input = Input(shape=(1,), name='aux_input')
    
    # computer vision model for 3-banded radar image
    x = Conv2D(64*3, kernel_size=4, padding='same', activation='relu')(radar_input)
    x = MaxPooling2D(pool_size=2)(x)
    x = Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(.25)(x)
    for _ in range(2):
        x = Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
        x = MaxPooling2D(pool_size=2)(x)
        x = Dropout(.15)(x)
    radar_output = Flatten()(x)
    
    # add auxiliary input and combine into concatenate layer
    combined_inputs = concatenate([radar_output, aux_input])
    
    # feed both into fully-connected layers
    x = Dense(1024, activation='relu')(combined_inputs)
    x = Dropout(.2)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(256, activation='relu')(x)
    
    # final output
    last_act = 'sigmoid' if iceberg_predict else None
    last_name = 'is_iceberg' if iceberg_predict else 'inc_angle'
    output = Dense(1, activation=last_act, name=last_name)(x)
    
    # model definition and summary
    model = Model(inputs=[radar_input, aux_input], outputs=output)
    model.summary()
    plot_model(model, to_file='cnn-{}.png'.format('iceberg' if iceberg_predict else 'angle'))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
radar_image (InputLayer)         (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
conv2d_49 (Conv2D)               (None, 75, 75, 128)   6272        radar_image[0][0]                
____________________________________________________________________________________________________
max_pooling2d_49 (MaxPooling2D)  (None, 25, 25, 128)   0           conv2d_49[0][0]                  
____________________________________________________________________________________________________
conv2d_50 (Conv2D)               (None, 25, 25, 128)   147584      max_pooling2d_49[0][0]           
___________________________________________________________________________________________

In [62]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=5)
save_best = ModelCheckpoint(filepath=os.path.join('saved_models', 
                                                  'cnn.inc_angle.best.weights.hdf5'), 
                            save_best_only=True, save_weights_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

In [63]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [64]:
batch_size = 32
epochs = 100


def img_and_aux_gen(datagen, tensors, aux_input, targets):
    # https://www.kaggle.com/sinkie/keras-data-augmentation-with-multiple-inputs
    img_gen = datagen.flow(x=tensors, y=targets, batch_size=batch_size, seed=42)
    aux_gen = datagen.flow(x=np.zeros_like(tensors), y=aux_input, batch_size=batch_size, seed=42)
    while True:
        x1, y = img_gen.next()
        _, x2 = aux_gen.next()
        yield ([x1, x2], y)
        

gen_flow = img_and_aux_gen(datagen, train_tensors, train_is_iceberg, train_targets)

model.fit_generator(gen_flow, callbacks=[early_stop, save_best, reduce_lr],
                    steps_per_epoch=len(train_targets)/batch_size, 
                    epochs=epochs,
                    validation_data=([valid_tensors, valid_is_iceberg], valid_targets))

Epoch 1/100


 1/34 [..............................] - ETA: 79s - loss: 1520.1907 - mean_absolute_error: 38.8341

 2/34 [>.............................] - ETA: 57s - loss: 1415.6057 - mean_absolute_error: 37.4253

 3/34 [=>............................] - ETA: 47s - loss: 1123.9695 - mean_absolute_error: 32.4617

 4/34 [==>...........................] - ETA: 42s - loss: 995.2419 - mean_absolute_error: 28.9306 

...

32/34 [==========================>...] - ETA: 2s - loss: 11.5226 - mean_absolute_error: 2.6481

33/34 [===========================>..] - ETA: 1s - loss: 11.8457 - mean_absolute_error: 2.6858

34/34 [============================>.] - ETA: 0s - loss: 11.8678 - mean_absolute_error: 2.6832

Epoch 00011: val_loss did not improve
35/34 [==============================] - 38s - loss: 12.2488 - mean_absolute_error: 2.7186 - val_loss: 33707.5058 - val_mean_absolute_error: 183.3127


Epoch 00011: early stopping


In [67]:
valid_preds = model.predict([valid_tensors, valid_is_iceberg])

In [69]:
valid_targets

(368,)